# Curve-fit to estimate final dissipation

In [3]:
%%capture
%run table.ipynb

In [4]:
path = paths_sim["noise_c20nh1920Bu0.5"]
params = ParamContainer(path_file=path+'/params_simul.xml')
print(_k_diss(params), _k_f(params), _k_d(params), _k_max(params))

43.481953290870486 0.7539822368615504 1.066291905158008 107.2330292425316


In [5]:
params.preprocess.viscosity_const * 2.5

1.9625000000000001

In [6]:
%matplotlib ipympl
def get_teps(short_name):
    path = paths_sim[short_name]
    d = SpatialMeansSW1L._load(path)
    t = d['t']
    eps = d['epsK'] + d ['epsA']
    
    idx = _index_where(eps, eps.max())
    
    return t, eps, idx

def plot_eps(short_name):
    t, eps,idx = get_teps(short_name)
    plt.plot(t, eps, label=short_name)
    plt.text(t[idx], eps[idx], short_name)
    # plt.legend()
    
## Some extreme cases
#plot_eps('noise_c400nh7680Buinf')
#plot_eps('vortex_grid_c100nh1920Bu2.0efr1.00e+00')
#plot_eps('noise_c20nh7680Buinf')
#plot_eps('vortex_grid_c100nh1920Bu1.0efr1.00e+00')
#plot_eps('noise_c40nh7680Buinf')
#plot_eps('vortex_grid_c100nh1920Buinfefr1.00e+02')

## Lognorm like
# plot_eps('vortex_grid_c20nh1920Buinfefr1.00e+01')
# plot_eps('vortex_grid_c100nh1920Buinfefr1.00e+02')
# plot_eps('vortex_grid_c100nh1920Bu4.0efr1.00e+02')
# plot_eps('vortex_grid_c400nh1920Buinfefr1.00e+02')

#for short in df_vort['short name']:
#    plot_eps(short)

# plot_eps('noise_c400nh3840Buinf')

In [7]:
%matplotlib
t, eps, idx = get_teps("noise_c400nh3840Buinf")
plt.plot(t, np.tanh(t/10))
plt.plot(t, np.tanh((t/10)**4))
plt.show()

Using matplotlib backend: module://ipympl.backend_nbagg


/scratch/avmo/opt/fluidmeta/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

Working version:

In [10]:
%matplotlib ipympl
from scipy.optimize import curve_fit
from scipy import stats

def f(x, amptan, ttan, amplog, sigma):
    return  amptan * np.tanh(2 * (x/ttan)**4) + amplog * stats.lognorm.pdf(x, scale=np.exp(ttan), s=sigma) 
    # return (c + d/(np.sinh(x+t))**2) * np.tanh(2 * (x/t)**4)
    # return (c + d/(x+t)**2)* np.tanh(2 * (x/t)**4)
    
short = 'vortex_grid_c100nh1920Buinfefr1.00e+02'
short = 'vortex_grid_c20nh1920Buinfefr1.00e+01'
short = 'vortex_grid_c400nh1920Buinfefr1.00e+02'
# short = 'noise_c20nh7680Buinf'
# short = 'vortex_grid_c20nh960Buinfefr1.00e+00'
t, eps,_ = get_teps(short)

guesses = {
    'amptan': np.median(eps),
    'ttan': t[eps==eps.max()],
    'amplog': eps.max(),
    'sigma': eps.std()
}
guesses = np.array(list(guesses.values()))
bounds = (0, guesses * 1.5)

# popt, pcov = curve_fit(f, t, eps)
# popt, pcov = curve_fit(f, t, eps, sigma=1./t)
popt, pcov = curve_fit(f, t, eps, guesses)

# popt, pcov = curve_fit(f, t, eps, guesses, bounds=bounds, method="trf")

plot(t, eps, label='original')
plot(t, f(t, *popt), label='curve fit')
plot(t, np.median(eps) * np.ones_like(eps), 'g', label='median_all')

plot(t, np.median(eps[t>40]) * np.ones_like(eps), 'r:', label='median')
plot(t, np.mean(eps[t>40]) * np.ones_like(eps), 'r--', label='mean')

df = df_vort if  'vortex' in short else df_noise
# eps_chosen = get_row(df, 'short name', short)['$\epsilon$'].iloc[0]

# plot(t, eps_chosen * np.ones_like(eps), 'k', label='chosen')
# plot(t, popt[2] * stats.lognorm.pdf(t, *popt[-2:]), label='lognorm')
legend()

/scratch/avmo/opt/fluidmeta/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()